In [ ]:
from langgraph.graph import StateGraph, END
import random
from typing_extensions import TypedDict
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import os
from typing import List, Dict, Any
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers.json import JsonOutputParser


# Estado

In [73]:
class State(TypedDict):
    user_input: str
    modo: str
    respuestas: List[str]
    pregunta_idx: int
    feedback: Dict[str, Any]
    nivel: str
    fortalezas: List[str]
    debilidades: List[str]
    puntaje_promedio: float
    detalle: List[Dict[str, Any]]
    preguntas_seleccionadas: List[Dict[str, Any]]
    temas: List[str]
    subtemas : Dict[str, Any]
    tema_actual : int

# Preguntas Quiz

In [74]:
quiz_preguntas = {
    "basico": [
        {"tema": "Variables aleatorias", "pregunta": "¿Qué es una variable aleatoria?"},
        {"tema": "Variables aleatorias", "pregunta": "¿Cuál es la diferencia entre una variable aleatoria discreta y continua?"},
        {"tema": "Probabilidad", "pregunta": "¿Qué es la probabilidad clásica?"},
        {"tema": "Probabilidad", "pregunta": "¿Qué es la probabilidad frecuentista?"},
        {"tema": "Distribuciones simples", "pregunta": "¿Qué es una distribución uniforme?"},
        {"tema": "Distribuciones simples", "pregunta": "¿Qué caracteriza a una distribución de probabilidad discreta?"},
        {"tema": "Eventos", "pregunta": "¿Qué es un evento en probabilidad?"},
        {"tema": "Eventos", "pregunta": "¿Qué significa que dos eventos sean mutuamente excluyentes?"}
    ],
    "intermedio": [
        {"tema": "Desviación estándar", "pregunta": "¿Qué es la desviación estándar?"},
        {"tema": "Desviación estándar", "pregunta": "¿Cómo se interpreta una desviación estándar alta o baja?"},
        {"tema": "Medidas de dispersión", "pregunta": "¿Qué es la varianza?"},
        {"tema": "Medidas de dispersión", "pregunta": "¿Cómo se relacionan la varianza y la desviación estándar?"},
        {"tema": "Estadística descriptiva", "pregunta": "¿Qué es la media aritmética?"},
        {"tema": "Estadística descriptiva", "pregunta": "¿Qué diferencia hay entre la media y la mediana?"},
        {"tema": "Distribuciones", "pregunta": "¿Qué es una distribución normal?"},
        {"tema": "Distribuciones", "pregunta": "¿Qué es una distribución sesgada y cómo se identifica?"}
    ],
    "avanzado": [
        {"tema": "Probabilidad conjunta", "pregunta": "¿Cómo se calcula la probabilidad conjunta de dos eventos independientes?"},
        {"tema": "Probabilidad conjunta", "pregunta": "¿Qué diferencia hay entre probabilidad conjunta y probabilidad condicional?"},
        {"tema": "Teorema de Bayes", "pregunta": "Explica el teorema de Bayes con un ejemplo."},
        {"tema": "Teorema de Bayes", "pregunta": "¿Cómo se aplica el teorema de Bayes en problemas médicos?"},
        {"tema": "Distribuciones avanzadas", "pregunta": "¿Qué es una distribución binomial?"},
        {"tema": "Distribuciones avanzadas", "pregunta": "¿Qué es una distribución de Poisson y en qué casos se utiliza?"},
        {"tema": "Inferencia", "pregunta": "¿Qué es una estimación puntual en inferencia estadística?"},
        {"tema": "Inferencia", "pregunta": "¿Qué diferencia hay entre una estimación puntual y un intervalo de confianza?"}
    ]
}


# Configuracion LLM

In [5]:
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [21]:
llm = ChatOpenAI(model="o4-mini")

In [12]:
# prueba llm
respuesta = llm.invoke("¿Qué es una variable aleatoria?")
respuesta

AIMessage(content='Una variable aleatoria es un concepto de probabilidad que sirve para modelar cuantitativamente el azar. De forma más precisa:\n\n1. Espacio muestral (Ω): conjunto de todos los posibles resultados de un experimento aleatorio (por ejemplo, caras o cruces al lanzar una moneda, o los valores de un dado).\n\n2. Variable aleatoria (X): función que asigna a cada resultado ω∈Ω un número real X(ω). Ese número es la “observación” o “valor” que toma la variable cuando ocurre ω.\n\nTipos principales  \n• Discreta: toma un conjunto contable de valores (por ejemplo, X = número de caras al lanzar 3 monedas: {0,1,2,3}). Se describe por su función de masa de probabilidad pX(k)=P(X=k).  \n• Continua: puede tomar cualquier valor en uno o varios intervalos de la recta. Se describe por su función de densidad fX(x), de modo que P(a≤X≤b)=∫ab fX(x)dx.\n\nFunciones asociadas  \n- Función de distribución acumulada (FDA): FX(x)=P(X≤x).  \n- Media o esperanza: E[X]=∑k k·pX(k) (discreto) ó ∫ x·f

# Nodos

## nodo_inicio

In [78]:
## nodo
def classify_node(state):
    user_input = state["user_input"]
    if not user_input:
            return {"modo": ""}
    prompt = (
        "Clasifica la siguiente intención del usuario SOLO como 'guiado' o 'libre'. "
        "Si el usuario quiere que lo guíes paso a paso, responde 'guiado'. "
        "Si solo quiere una respuesta directa, responde 'libre'. "
        f"Intención del usuario: {user_input}\n"
        "Respuesta:"
    )
    respuesta = llm.invoke(prompt).content.strip().lower()
    if "guiado" in respuesta:
        modo = "guiado"
    elif "libre" in respuesta:
        modo = "libre"
    else:
        modo = "libre"
    return {"modo": modo}

In [79]:
#edge
def transicion_inicio(state: State):
    if state.modo == "modo libre":
        return "quiz_nivel"
    else:
        return "libre"

In [80]:
estado_prueba = State(
    user_input="quiero que me guies paso a paso",
    modo="",
    respuestas=[],
    pregunta_idx=0,
    feedback={},
    nivel="basico",
    fortalezas=[],
    debilidades=[],
    puntaje_promedio=0.0,
    detalle=[],
    preguntas_seleccionadas=[]
)
classify_node(estado_prueba)

{'modo': 'guiado'}

## nodo quiz_nivel

In [81]:
prompt_quiz = ChatPromptTemplate.from_messages([
    ("system",
     """Eres un experto en educación. Evalúa las siguientes respuestas del usuario a preguntas de probabilidad y estadística.
Para cada respuesta, califica de 0 a 5 (donde 0 es incorrecta y 5 es perfecta), explica brevemente la calificación.

Devuelve la respuesta SOLO en formato JSON con la siguiente estructura:
{{
  "resultados": [puntaje1, puntaje2, ...],
  "detalle": [
    {{
      "pregunta": "...",
      "respuesta": "...",
      "tema": "...",
      "puntaje": 0-5,
      "feedback": "..."
    }},
    ...
  ]
}}

Respuestas del usuario:
{respuestas_usuario}
""")
])

In [82]:
parser = JsonOutputParser()
chain_quiz = prompt_quiz | llm | parser

In [83]:
# nodo
def nodo_calificar(state):
    respuestas = state.get("respuestas", [])
    preguntas_seleccionadas = state.get("preguntas_seleccionadas", [])
    respuestas_usuario = []
    for idx, pregunta in enumerate(preguntas_seleccionadas):
        if idx < len(respuestas):
            respuestas_usuario.append({
                "pregunta": pregunta["pregunta"],
                "respuesta": respuestas[idx],
                "tema": pregunta["tema"]
            })
    prompt_str = prompt_quiz.format(respuestas_usuario=str(respuestas_usuario))
    data = chain_quiz.invoke(prompt_str)

    resultados = data.get("resultados", [])
    detalle = data.get("detalle", [])
    promedio = sum(resultados) / len(resultados) if resultados else 0
    promedio = round(promedio, 2)
    fortalezas = [d["tema"] for d in detalle if d["puntaje"] >= 4]
    debilidades = [d["tema"] for d in detalle if d["puntaje"] < 4]
    state["feedback"] = data
    state["fortalezas"] = list(set(state.get("fortalezas", []) + fortalezas))
    state["debilidades"] = list(set(state.get("debilidades", []) + debilidades))
    state["puntaje_promedio"] = promedio
    state["detalle"] = detalle
    return state

In [84]:
estado_prueba = State(
    user_input="quiero que me guies paso a paso",
    modo="modo guiado",
    # Lista de respuestas del usuario
    respuestas=["Python es un lenguaje de programación interpretado",
                "Un bucle for se usa para iterar sobre una secuencia",
                "Una variable es un espacio en memoria que almacena datos"],

    pregunta_idx=3,  # Indica que ya se respondieron 3 preguntas

    # Las preguntas que fueron seleccionadas y respondidas
    preguntas_seleccionadas=[
        {
            "pregunta": "¿Qué es Python?",
            "tema": "Fundamentos de Python",
            "respuesta_correcta": "Python es un lenguaje de programación interpretado de alto nivel"
        },
        {
            "pregunta": "¿Para qué se usa un bucle for?",
            "tema": "Estructuras de Control",
            "respuesta_correcta": "Un bucle for se utiliza para iterar sobre una secuencia de elementos"
        },
        {
            "pregunta": "¿Qué es una variable?",
            "tema": "Conceptos Básicos",
            "respuesta_correcta": "Una variable es un espacio en memoria que almacena un valor"
        }
    ],

    feedback={},  # Se llenará después de la calificación
    nivel="basico",
    fortalezas=[],  # Se llenará después de la calificación
    debilidades=[],  # Se llenará después de la calificación
    puntaje_promedio=0.0,  # Se calculará después de la calificación
    detalle=[]  # Se llenará después de la calificación
)

# Probar el nodo
estado_actualizado = nodo_calificar(estado_prueba)

# Imprimir los resultados actualizados
print("\nResultados actualizados:")
print(f"Puntaje promedio: {estado_actualizado['puntaje_promedio']}")
print(f"Fortalezas: {estado_actualizado['fortalezas']}")
print(f"Debilidades: {estado_actualizado['debilidades']}")
print(f"Detalle: {estado_actualizado['detalle']}")



Resultados actualizados:
Puntaje promedio: 5.0
Fortalezas: ['Fundamentos de Python', 'Conceptos Básicos', 'Estructuras de Control']
Debilidades: []
Detalle: [{'pregunta': '¿Qué es Python?', 'respuesta': 'Python es un lenguaje de programación interpretado', 'tema': 'Fundamentos de Python', 'puntaje': 5, 'feedback': 'Definición precisa y completa de Python como lenguaje interpretado.'}, {'pregunta': '¿Para qué se usa un bucle for?', 'respuesta': 'Un bucle for se usa para iterar sobre una secuencia', 'tema': 'Estructuras de Control', 'puntaje': 5, 'feedback': 'Explicación correcta y clara del propósito de un bucle for.'}, {'pregunta': '¿Qué es una variable?', 'respuesta': 'Una variable es un espacio en memoria que almacena datos', 'tema': 'Conceptos Básicos', 'puntaje': 5, 'feedback': 'Descripción exacta de una variable y su función en programación.'}]


## nodo plan_estudio

In [85]:
def nodo_plan_estudio(state: State):
    # Corregido: usar [] en lugar de .
    debilidades = state["debilidades"]
    debilidades_str = ", ".join(debilidades)
    print(f"Debilidades: {debilidades_str}")

    prompt_plan = ChatPromptTemplate.from_template(
        """
    Eres un tutor experto en estadística y probabilidad.
    El estudiante tiene el nivel: {nivel}.
    Sus debilidades principales son: {debilidades}.

    Crea un plan de estudio personalizado y devuélvelo SOLO en formato JSON con EXACTAMENTE esta estructura:
    {{
        "plan_estudio": {{
            "tema1": {{
                "nombre": "Nombre del Tema 1",
                "subtemas": [
                    "Subtema 1.1",
                    "Subtema 1.2",
                    "Subtema 1.3",
                    "Subtema 1.4"
                ]
            }},
            "tema2": {{
                "nombre": "Nombre del Tema 2",
                "subtemas": [
                    "Subtema 2.1",
                    "Subtema 2.2",
                    "Subtema 2.3",
                    "Subtema 2.4"
                ]
            }},
            "tema3": {{
                "nombre": "Nombre del Tema 3",
                "subtemas": [
                    "Subtema 3.1",
                    "Subtema 3.2",
                    "Subtema 3.3",
                    "Subtema 3.4"
                ]
            }}
        }}
    }}

    Los temas deben enfocarse en las debilidades mencionadas.
    Cada tema DEBE tener exactamente 4 subtemas.
    IMPORTANTE: Devuelve SOLO el JSON, sin texto adicional.
    """
    )

    parser = JsonOutputParser()
    chain = prompt_plan | llm | parser

    # Corregido: usar [] para acceder a los elementos
    respuesta = chain.invoke({
        "nivel": state["nivel"],
        "debilidades": debilidades_str
    })

    plan = respuesta["plan_estudio"]
    state_actualizado = state.copy()

    # Actualizar el estado con la información del plan
    temas = []
    subtemas = {}

    for tema_key, tema_data in plan.items():
        nombre_tema = tema_data["nombre"]
        temas.append(nombre_tema)
        subtemas[nombre_tema] = tema_data["subtemas"]

    state_actualizado["temas"] = temas
    state_actualizado["subtemas"] = subtemas
    state_actualizado["tema_actual"] = 0

    # Imprimir el plan de forma legible
    print("\nPlan de estudio personalizado:")
    for i, tema in enumerate(temas, 1):
        print(f"\n{i}. {tema}")
        for subtema in subtemas[tema]:
            print(f"   • {subtema}")

    return state_actualizado

In [86]:
# Ejemplo de uso
estado_inicial = State(
    user_input="",
    modo="guiado",
    respuestas=[],
    pregunta_idx=0,
    feedback={},
    nivel="intermedio",
    fortalezas=[],
    debilidades=["probabilidad condicional", "distribuciones", "teorema de Bayes"],
    puntaje_promedio=0.0,
    detalle=[],
    preguntas_seleccionadas=[],
    temas=[],
    subtemas={},
    tema_actual=0
)

In [87]:
nodo_plan_estudio(estado_inicial)

Debilidades: probabilidad condicional, distribuciones, teorema de Bayes

Plan de estudio personalizado:

1. Probabilidad Condicional
   • Definición y fórmula de P(A|B)
   • Regla de la multiplicación
   • Teorema de la probabilidad total
   • Ejemplos prácticos con árboles de probabilidad

2. Distribuciones de Probabilidad
   • Distribuciones discretas: Binomial y Poisson
   • Distribuciones continuas: Uniforme y Normal
   • Función de densidad y función de distribución acumulada
   • Cálculo de esperanza y varianza

3. Teorema de Bayes
   • Derivación de la fórmula de Bayes
   • Priori, verosimilitud y posteriori
   • Aplicaciones en diagnósticos médicos
   • Resolución de problemas complejos


{'user_input': '',
 'modo': 'guiado',
 'respuestas': [],
 'pregunta_idx': 0,
 'feedback': {},
 'nivel': 'intermedio',
 'fortalezas': [],
 'debilidades': ['probabilidad condicional',
  'distribuciones',
  'teorema de Bayes'],
 'puntaje_promedio': 0.0,
 'detalle': [],
 'preguntas_seleccionadas': [],
 'temas': ['Probabilidad Condicional',
  'Distribuciones de Probabilidad',
  'Teorema de Bayes'],
 'subtemas': {'Probabilidad Condicional': ['Definición y fórmula de P(A|B)',
   'Regla de la multiplicación',
   'Teorema de la probabilidad total',
   'Ejemplos prácticos con árboles de probabilidad'],
  'Distribuciones de Probabilidad': ['Distribuciones discretas: Binomial y Poisson',
   'Distribuciones continuas: Uniforme y Normal',
   'Función de densidad y función de distribución acumulada',
   'Cálculo de esperanza y varianza'],
  'Teorema de Bayes': ['Derivación de la fórmula de Bayes',
   'Priori, verosimilitud y posteriori',
   'Aplicaciones en diagnósticos médicos',
   'Resolución de pr

# Nodo explicacion (RAG)

## Toca hacer que solo explique un tema que lo devuelva en formato mark down, y que luego la UI le vaya pasando 1 a 1 los temas segun avanza el ciclo

In [7]:
ejemplo_documentos_rag = [
    """
    La probabilidad condicional es la probabilidad de que ocurra un evento A,
    sabiendo que también ha ocurrido otro evento B. Se denota como P(A|B) y
    se calcula como: P(A|B) = P(A∩B) / P(B)

    La regla de la cadena establece que: P(A∩B) = P(A|B) × P(B)
    """,

    """
    El Teorema de Bayes es una herramienta fundamental que relaciona las
    probabilidades condicionales de dos eventos. Su fórmula es:
    P(A|B) = P(B|A) × P(A) / P(B)

    Este teorema es especialmente útil cuando tenemos probabilidades previas
    y queremos actualizar nuestro conocimiento con nueva información.
    """,

    """
    Los árboles de probabilidad son herramientas visuales que ayudan a
    resolver problemas de probabilidad condicional. Cada rama representa
    un evento y sus probabilidades asociadas.
    """
]

In [17]:
from langchain.vectorstores import Chroma  # o cualquier otra base de vectores
from langchain.embeddings import OpenAIEmbeddings  # o tus embeddings preferidos

# Configuración del RAG
embeddings = OpenAIEmbeddings()
vectorstore = Chroma(embedding_function=embeddings)

# Cargar los documentos (solo una vez)
vectorstore.add_texts(ejemplo_documentos_rag)

# Crear el retriever estándar de LangChain
retriever = vectorstore.as_retriever()

In [23]:
class AgenteExplicador:
    def __init__(self, rag_retriever, llm):
        self.retriever = rag_retriever
        self.llm = llm

    def construir_query(self, tema, subtemas):
        subtemas_str = "\n".join([f"- {s}" for s in subtemas])
        return f"""
        Necesito información detallada sobre {tema}, específicamente:
        {subtemas_str}
        Incluye definiciones, fórmulas y ejemplos prácticos.
        """

    def construir_prompt_llm(self, tema, subtemas, contexto_rag):
        estructura = ""
        for i, subtema in enumerate(subtemas, 1):
            estructura += f"""{i}. {subtema}
   - Definición
   - Fórmulas clave (si aplica)
   - Ejemplo práctico

"""
        return f"""
Eres un tutor experto en estadística y probabilidad.

CONTEXTO RECUPERADO:
{contexto_rag}

TAREA:
Explica el tema "{tema}" de manera clara y estructurada.

ESTRUCTURA REQUERIDA:
{estructura}
INSTRUCCIONES:
- Usa un lenguaje claro y accesible
- Incluye fórmulas matemáticas cuando sea necesario
- Proporciona ejemplos sencillos para cada concepto
- Relaciona los conceptos entre sí cuando sea relevante
"""

    def explicar_tema(self, tema, subtemas):
        # 1. Construir y ejecutar query para RAG
        query = self.construir_query(tema, subtemas)
        print("\nQuery para RAG:")
        print(query)

        # 2. Obtener documentos relevantes usando el retriever estándar
        documentos = self.retriever.get_relevant_documents(query)
        contexto = "\n\n".join([doc.page_content for doc in documentos])
        print("\nDocumentos recuperados:", len(documentos))

        # 3. Construir prompt para el LLM
        prompt_final = self.construir_prompt_llm(tema, subtemas, contexto)
        print("\nPrompt para LLM construido")

        # 4. Generar explicación
        explicacion = self.llm.invoke(prompt_final)
        return explicacion


In [24]:
tema_ejemplo = "Probabilidad condicional"
subtemas_ejemplo = ["Teorema de Bayes", "probabilidad"]

agente = AgenteExplicador(retriever, llm)
explicacion = agente.explicar_tema(tema_ejemplo, subtemas_ejemplo)
print("\nExplicación generada:")
print(explicacion)


Query para RAG:

        Necesito información detallada sobre Probabilidad condicional, específicamente:
        - Teorema de Bayes
- probabilidad
        Incluye definiciones, fórmulas y ejemplos prácticos.
        

Documentos recuperados: 4

Prompt para LLM construido

Explicación generada:
content='1. Teorema de Bayes  \n   - Definición  \n     El Teorema de Bayes permite “invertir” una probabilidad condicional. Nos dice cómo actualizar la probabilidad de una hipótesis A (por ejemplo, tener una enfermedad) cuando disponemos de nueva evidencia B (por ejemplo, un resultado positivo en un test).  \n   - Fórmulas clave  \n     1) P(A|B) = [P(B|A) · P(A)] / P(B)  \n     2) Si las hipótesis A₁, A₂,…, Aₙ son exhaustivas y mutuamente excluyentes,  \n        P(Aᵢ|B) = [P(B|Aᵢ) · P(Aᵢ)] / Σⱼ P(B|Aⱼ)·P(Aⱼ)  \n   - Ejemplo práctico  \n     Supongamos un test de cáncer con:  \n       • Prevalencia P(C) = 1% = 0,01  \n       • Sensibilidad P(Pos|C) = 0,99  \n       • Especificidad P(Neg|¬C) = 0

# Nodo respuestas (RAG)

# Tools